In [32]:
import json

In [33]:
with open("qpsr.json") as f:
    data = json.load(f)

In [34]:
res = {}
for item in data:
    if not "pages" in item:
        print(f"Missing pages: {item['title']} ({item['year']})")
        pages = "?-?"
    else:
        pages = item["pages"]
    year = item["year"]
    if not "volume" in item:
        print(f"Missing volume: {item['title']} ({item['year']})")
        vol = "??"
    else:
        vol = item["volume"]
    if not "edition" in item:
        print(f"Missing edition: {item['title']} ({item['year']})")
        ed = "??"
    else:
        ed = item["edition"]
    if not "-" in pages:
        if pages.isdigit():
            start = end = pages
        else:
            raise IOError(f"No '-' in pages: {pages} ({key})")
    else:
        start, end = pages.split("-")
    if item["title"] == ".":
        print(item["pdf"])
    key = f"{year}_{vol}_{ed}"
    if not key in res:
        res[key] = [{"start": start, "end": end}]
    else:
        res[key].append({"start": start, "end": end})

http://www.speech.kth.se/prod/publications/files/3605.pdf
http://www.speech.kth.se/prod/publications/files/3607.pdf
http://www.speech.kth.se/prod/publications/files/3579.pdf
http://www.speech.kth.se/prod/publications/files/3586.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_121-124.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_033-036.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_049-052.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_053-056.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_061-064.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_077-080.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_093-096.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2007/2007_50_1_097-100.pdf
http://www.speech.kth.se/prod/publications/files/qpsr/2006/2006_48_1_035-043.pdf
http://www.speech.kth.se/prod/publicati